**Import Libraries for Quantum Computing and Noise Modeling with `quairkit`**

In this code cell, we import the necessary libraries and components to build, manipulate, and analyze quantum circuits using the `quairkit` package, along with additional tools from standard Python libraries. This setup enables both classical and quantum operations, particularly focusing on quantum information processing and noise modeling.

**Key Imports and Their Uses**:

1. **Time and Random Utilities**:
   - **`time`**: Imported from the standard library to measure execution time or introduce delays.
   - **`randint`**: Imported from the `random` module to generate random integers, which can be used for generating randomness in simulations.

2. **`quairkit` for Quantum Computing**:
   - **`quairkit`**: The core library used for quantum information operations, including circuit construction, quantum state management, and noise modeling.
   - **`torch`**: This is a library for deep learning, but in the context of `quairkit`, it may be used for tensor manipulations and handling quantum state representations efficiently.
   
3. **Circuit Construction and Manipulation**:
   - **`Circuit`** from `quairkit.circuit`: Used for constructing quantum circuits. This component is critical for defining the structure of quantum gates and qubits involved in an experiment.

4. **Quantum State Management**:
   - **`to_state`** from `quairkit.core`: Converts a given representation into a quantum state, helpful for initializing or analyzing different quantum states.
   - **`bell_state`, `one_state`, `zero_state`** from `quairkit.database`:
     - **`bell_state`**: Represents a Bell state, which is a maximally entangled two-qubit state, often used to study quantum entanglement.
     - **`one_state`, `zero_state`**: These represent the |1⟩ and |0⟩ states, respectively, used frequently in initializing qubits for quantum circuits.

5. **Quantum Noise Modeling**:
   - **`AmplitudeDamping`, `BitFlip`** from `quairkit.operator`: Models specific types of quantum noise.
     - **`AmplitudeDamping`**: Represents amplitude damping noise, simulating energy loss in a quantum system.
     - **`BitFlip`**: Represents bit-flip noise, where qubits randomly flip between |0⟩ and |1⟩ due to noise, mimicking real-world errors in quantum systems.

6. **Quantum Information Tools**:
   - **`NKron`, `dagger`, `trace`** from `quairkit.qinfo`:
     - **`NKron`**: Performs the Kronecker product (tensor product) over multiple matrices, which is fundamental for creating composite quantum systems.
     - **`dagger`**: Computes the conjugate transpose (Hermitian conjugate) of an operator, essential for certain quantum operations.
     - **`trace`**: Computes the trace of a matrix, often used to extract information or reduce the system's state representation.

7. **Statistics**:
   - **`mean`** from `statistics`: Computes the average of a set of numbers, useful for summarizing results, such as the average fidelity or performance metrics over multiple quantum simulations.

These imports establish a complete toolkit for quantum computing simulations, focusing on circuit construction, state manipulation, and noise modeling using `quairkit`, complemented by standard utilities from Python.


In [ ]:
import time  # Measure execution time or introduce delays
from random import randint  # Generate random integers for simulations or random testing

# Import `quairkit`, a specialized quantum computing library
import quairkit
import torch  # Useful for tensor operations, potentially for managing quantum state vectors

# Import classes for building and manipulating quantum circuits
from quairkit.circuit import Circuit

# Import core functionality for converting different types to quantum states
from quairkit.core import to_state

# Import standard quantum states from `quairkit`'s database
from quairkit.database import bell_state, one_state, zero_state

# Import quantum noise operators for modeling noise in circuits
from quairkit.operator import AmplitudeDamping, BitFlip

# Import quantum information tools for matrix operations
from quairkit.qinfo import NKron, dagger, trace

# Import statistical utilities to calculate averages
from statistics import mean


**Configure `quairkit` Settings for Precision and Reproducibility**

This cell sets key configuration parameters for the `quairkit` library to ensure precision in numerical operations and reproducibility in stochastic processes:

1. **Set Numerical Precision**

2. **Set Random Seed for Reproducibility**

In [ ]:
# Set complex number precision for `quairkit`
quairkit.set_dtype("complex128")

# Set a random seed for reproducibility
seed = randint(0, int(1e6))
quairkit.set_seed(seed)

**Define the Choi Representation for Two Quantum Channels**

This code calculates the Choi representation of two different quantum noise channels, which is a standard way to describe quantum operations and analyze their behavior:

1. **Parameters**:
   - **`gamma = 0.67`**: Damping factor for amplitude damping channel.
   - **`eta = 0.87`**: Parameter controlling the strength of the bit-flip noise channel.

2. **Quantum Channels**:
   - **`AmplitudeDamping(gamma).choi_repr`**: Creates the Choi matrix representation for an amplitude damping channel with damping factor `gamma`.
   - **`BitFlip(1 - eta).choi_repr`**: Creates the Choi representation for a bit-flip channel with a flip probability of `1 - eta`.

These representations are useful for studying the properties of the channels and for distinguishing between different noise models.


In [ ]:
# Parameters for quantum channels
gamma = 0.67
eta = 0.87

# Choi representations of the two quantum channels to be distinguished
channel_choi_zero = AmplitudeDamping(gamma).choi_repr
channel_choi_one = BitFlip(1 - eta).choi_repr


**Create a Quantum Circuit for Channel Discrimination**

This cell defines a quantum circuit setup for a task called "Channel Discrimination," where quantum channels are analyzed using ancillary systems and multiple qudits:

1. **Task and Parameters**:
   - **`task_name = "Channel_discrimination"`**: Identifies the task as a channel discrimination problem.
   - **`num_slots = 2`**: Number of quantum channel slots to distinguish.
   - **`ancilla_dimension = 32`**: The dimension of the ancilla system used in the circuit.
   - **`channel_dimension = 2`**: The dimension of each quantum channel.

2. **System Dimensions**:
   - **`system_dim`**: Combines ancillary and channel dimensions to define the total structure of the quantum system, including ancillary qudits and channel qudits.

3. **Circuit Definition**:
   - **`cir = Circuit(system_dim=system_dim)`**: Initializes a circuit with the defined dimensions.
   - **`cir.universal_qudits([0, 2 * index + 1])`**: Applies universal operations on qudits, where the first qudit (ancilla) interacts with each channel slot.

This structure allows the circuit to use ancillary dimensions to help distinguish between quantum channels in multiple slots.


In [ ]:
# Task name and parameters for channel discrimination
task_name = "Channel_discrimination"
num_slots = 2
ancilla_dimension = 32
channel_dimension = 2

# Define system dimensions: ancilla + channel slots
system_dim = [ancilla_dimension] + [channel_dimension] * (2 * num_slots + 1)

# Initialize the quantum circuit with the specified dimensions
cir = Circuit(system_dim=system_dim)

# Apply universal operations to the ancilla and channel qudits
for index in range(num_slots + 1):
    cir.universal_qudits([0, 2 * index + 1])


**Initialize the Input State for Channel Discrimination**

This code initializes the input state for the quantum circuit, combining ancillary and channel states to form the full quantum system used in the channel discrimination task:

1. **State Initialization**:
   - **`zero_state(2, system_dim[:2])`**: Creates the initial state for the ancilla as the |0⟩ state, with a specified system dimension of the first two qudits (ancilla dimension).
   - **`bell_state(2, [channel_dimension] * 2)`**: Initializes each channel slot in a maximally entangled Bell state between two qudits, with a dimension of `channel_dimension`.

2. **Tensor Product of States**:
   - **`NKron(...)`**: Uses the Kronecker product to combine the ancilla state and the Bell states from each channel slot into a complete state vector for the system.

3. **Create the Input State**:
   - **`to_state(...)`**: Converts the resulting tensor product into a proper quantum state representation for use in the circuit, with the full `system_dim`.

This combined state is used as the starting point for the quantum discrimination process, utilizing both ancilla and entangled states to enhance discrimination power.


In [ ]:
# Initialize the input state for the channel discrimination task
input_state = to_state(
    NKron(
        zero_state(2, system_dim[:2]).ket,  # Ancilla initialized to |0⟩ state
        *[bell_state(2, [channel_dimension] * 2).ket] * num_slots  # Multiple Bell states for each channel slot
    ),
    system_dim=system_dim  # Define the full system dimensions
)


**Define Projection Operators for Channel Discrimination**

This code defines projection operators that will be used to distinguish between two quantum channels, which is a crucial part of the channel discrimination task:

1. **Calculate Choi Dimension**:
   - **`choi_dim = torch.prod(torch.tensor(system_dim[1:-1]))`**:
     - Computes the product of all channel-related dimensions, excluding the ancillary dimension, to determine the Choi representation's dimension. This helps define the correct size for projection operators.

2. **Define Projection Operators**:
   - **`proj_zero`**:
     - **`torch.kron(torch.eye(choi_dim), zero_state(1, channel_dimension).bra)`**:
     - Constructs the projection operator for detecting a zero state in the channel. The identity matrix (`torch.eye`) applies to the other dimensions, while the zero state (`bra`) focuses on the channel state.
   - **`proj_one`**:
     - Similar to `proj_zero`, but uses `one_state` to project onto the |1⟩ channel state.

These projection operators help extract information from the quantum channels, distinguishing between different possible outcomes.


In [ ]:
# Calculate Choi dimension by multiplying relevant channel dimensions
choi_dim = torch.prod(torch.tensor(system_dim[1:-1]))

# Define projection operators for the channel discrimination task
proj_zero = torch.kron(
    torch.eye(choi_dim),  # Identity for the Choi space
    zero_state(1, channel_dimension).bra  # Projection onto |0⟩ state
)
proj_one = torch.kron(
    torch.eye(choi_dim),  # Identity for the Choi space
    one_state(1, channel_dimension).bra  # Projection onto |1⟩ state
)


**Define Loss Function for Channel Discrimination Task**

This function defines a loss function for training a quantum circuit to distinguish between two quantum channels. The goal is to minimize the loss, improving the accuracy of channel discrimination.

1. **Input**:
  - **`circuit: Circuit`**: The quantum circuit that is being optimized.

2. **Calculate Output State**:
  - **`output_state = circuit(input_state).trace([0]).density_matrix`**:
    - Applies the circuit to the input state, then traces out the ancilla qudit to focus on the remaining system.
  - **`output_state *= 2**num_slots`**:
    - Scales the output state by \(2^{\text{num\_slots}}\) to normalize it.

3. **Construct Choi Representations**:
  - **`choi_zero` and `choi_one`**:
    - Construct the respective Choi representations using the projection operators (`proj_zero`, `proj_one`) applied to the output state.

4. **Calculate Loss**:
  - **Loss Formula**:
    - Combines the trace values of `choi_zero` and `choi_one` with their respective Choi representations (`channel_choi_zero`, `channel_choi_one`).
    - The final loss is designed to be minimized as the circuit becomes more effective at distinguishing the two channels.

This loss function guides the training of the quantum circuit to achieve better discrimination between the two channels by optimizing the output state alignment.


In [ ]:
def loss_func(circuit: Circuit) -> torch.Tensor:
    # Apply the circuit to the input state and trace out the ancilla (qudit 0)
    output_state = circuit(input_state).trace([0]).density_matrix
    # Normalize the output state by scaling it
    output_state *= 2**num_slots

    # Construct Choi representations for channel discrimination
    choi_zero = proj_zero @ output_state @ dagger(proj_zero)
    choi_one = proj_one @ output_state @ dagger(proj_one)

    # Calculate the fidelity of the Choi representations
    zero_fid = trace(choi_zero @ NKron(*[channel_choi_zero] * num_slots))
    one_fid = trace(choi_one @ NKron(*[channel_choi_one] * num_slots))

    # Calculate and return the loss value
    return 1 - (zero_fid + one_fid).real / 2

**Training Loop for Channel Discrimination Using Gradient Descent**

This code trains a quantum circuit to minimize a loss function for a channel discrimination task, utilizing an optimizer and a learning rate scheduler to iteratively improve the circuit's parameters.

1. **Initialization**:
   - **`time_list = []`**: An empty list to store the time taken for each training iteration.
   - **Optimizer and Scheduler**:
     - **`opt = torch.optim.Adam(...)`**: Initializes an Adam optimizer with a learning rate of `0.1` to adjust the circuit parameters.
     - **`scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(...)`**: Sets up a scheduler that reduces the learning rate if the loss does not improve, helping avoid local minima.

2. **Training Loop**:
   - **Iterations**:
     - **`NUM_ITR := 1000`**: The total number of iterations is set to 1000.
   - **Loss Calculation and Backpropagation**:
     - **`loss = loss_func(cir)`**: Computes the loss using the custom loss function.
     - **`loss.backward()`**: Backpropagates the computed loss to calculate gradients.
     - **`opt.step()`**: Updates the circuit parameters.
     - **`scheduler.step(loss)`**: Updates the learning rate based on the latest loss.
   - **Progress Monitoring**:
     - Every 100 iterations (or at the last iteration), prints the current state, including iteration number, fidelity, learning rate, and average time taken.
   - **Early Stopping**:
     - **`if scheduler.get_last_lr()[0] < 1e-8:`**: Stops the training if the learning rate falls below a small threshold, preventing further ineffective updates.

3. **Output**:
   - Prints the final fidelity after training completes.

The loop combines backpropagation with a dynamic learning rate strategy to optimize the circuit efficiently, logging progress periodically and breaking if further improvement becomes unlikely.


In [ ]:
# Initialize an empty list to store the time taken for each iteration
time_list = []

# Set up the optimizer and learning rate scheduler
opt = torch.optim.Adam(lr=0.1, params=cir.parameters())
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, "min")

# Training loop for the specified number of iterations
NUM_ITR = 1000
for itr in range(NUM_ITR):
    start_time = time.time()  # Record the start time of the iteration
    opt.zero_grad()  # Reset gradients
    loss = loss_func(cir)  # Compute the loss
    loss.backward()  # Backpropagate the loss
    opt.step()  # Update the parameters
    scheduler.step(loss)  # Update the learning rate
    loss = loss.item()  # Convert loss to a scalar
    time_list.append(time.time() - start_time)  # Record the time taken for the iteration

    # Print progress every 100 iterations or at the last iteration
    if itr % 100 == 0 or itr == NUM_ITR - 1:
        print(
            f"[{task_name} | num_slots: {num_slots} | ancilla_dimension: {ancilla_dimension} | seed: {seed}] "
            f"iter: {itr}, fidelity: {1 - loss:.8f}, "
            f"lr: {scheduler.get_last_lr()[0]:.2E}, avg_time: {mean(time_list):.4f}s"
        )
        time_list.clear()  # Clear the time list after printing

    # Break the loop if the learning rate is below a threshold
    if scheduler.get_last_lr()[0] < 1e-8:
        break

# Final fidelity after training
print(f"[{task_name} | num_slots: {num_slots} | ancilla_dimension: {ancilla_dimension} | seed: {seed}] final fidelity: {1 - loss:.8f}")


[Channel_discrimination | num_slots: 2 | ancilla_dimension: 32 | seed: 862121] iter: 0, fidelity: 0.49601787, lr: 1.00E-01, avg_time: 0.6796s
[Channel_discrimination | num_slots: 2 | ancilla_dimension: 32 | seed: 862121] iter: 100, fidelity: 0.84274412, lr: 1.00E-02, avg_time: 0.4617s
[Channel_discrimination | num_slots: 2 | ancilla_dimension: 32 | seed: 862121] iter: 200, fidelity: 0.84315641, lr: 1.00E-02, avg_time: 0.4100s
[Channel_discrimination | num_slots: 2 | ancilla_dimension: 32 | seed: 862121] iter: 300, fidelity: 0.84358419, lr: 1.00E-02, avg_time: 0.3788s
[Channel_discrimination | num_slots: 2 | ancilla_dimension: 32 | seed: 862121] iter: 400, fidelity: 0.84393767, lr: 1.00E-02, avg_time: 0.3808s
[Channel_discrimination | num_slots: 2 | ancilla_dimension: 32 | seed: 862121] iter: 500, fidelity: 0.84417199, lr: 1.00E-02, avg_time: 0.3786s
[Channel_discrimination | num_slots: 2 | ancilla_dimension: 32 | seed: 862121] iter: 600, fidelity: 0.84431793, lr: 1.00E-03, avg_time: 0.